Funktioniert aber nicht wirklich nachvollziehbar (Quelle: https://curiousily.com/posts/movie-review-sentiment-analysis-with-naive-bayes/)

Leicht besser als der andere aber weniger als 1%

In [99]:
import pandas as pd
import numpy as np
import re
from bs4 import BeautifulSoup
import math
import collections
from collections import defaultdict

from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords


import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

from sklearn.model_selection import train_test_split

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [100]:
train = pd.read_csv("labeledTrainData.csv", delimiter="\t")
test = pd.read_csv("testData.csv", delimiter="\t")

In [101]:
train.head(5)

,id,sentiment,review
0,5814_8,1,With all this stuff going down at the moment w...
1,2381_9,1,"\The Classic War of the Worlds\"" by Timothy Hi..."
2,7759_3,0,The film starts with a manager (Nicholas Bell)...
3,3630_4,0,It must be assumed that those who praised this...
4,9495_8,1,Superbly trashy and wondrously unpretentious 8...


In [102]:
test.head(5)

,id,review
0,12311_10,Naturally in a film who's main themes are of m...
1,8348_2,This movie is a disaster within a disaster fil...
2,5828_4,"All in all, this is a movie for kids. We saw i..."
3,7186_2,Afraid of the Dark left me with the impression...
4,12128_7,A very accurate depiction of small time mob li...


In [103]:
def clean(self, text):
    no_html = BeautifulSoup(text).get_text()
    clean = re.sub("[^a-z\s]+", " ", no_html, flags=re.IGNORECASE)
    return re.sub("(\s+)", " ", clean)

In [104]:
def tokenize(self, text):
    clean = self.clean(text).lower()
    stopwords_en = stopwords.words("english")
    return [w for w in re.split("\W+", clean) if not w in stopwords_en]

In [105]:
def fit(self, X, y):
    self.n_class_items = {}
    self.log_class_priors = {}
    self.word_counts = {}
    self.vocab = set()
    n = len(X)
    grouped_data = self.group_by_class(X, y)

    for c, data in grouped_data.items():
      self.n_class_items[c] = len(data)
      self.log_class_priors[c] = math.log(self.n_class_items[c] / n)
      self.word_counts[c] = defaultdict(lambda: 0)
      for text in data:
        counts = Counter(self.tokenizer.tokenize(text))
        for word, count in counts.items():
            if word not in self.vocab:
                self.vocab.add(word)
            self.word_counts[c][word] += count

In [106]:
def group_by_class(self, X, y):
  data = dict()
  for c in self.classes:
    data[c] = X[np.where(y == c)]
  return data

In [107]:
def predict(self, X):
    result = []
    for text in X:
      class_scores = {c: self.log_class_priors[c] for c in self.classes}
      words = set(self.tokenizer.tokenize(text))
      for word in words:
          if word not in self.vocab: continue
          for c in self.classes:
            log_w_given_c = self.laplace_smoothing(word, c)
            class_scores[c] += log_w_given_c
      result.append(max(class_scores, key=class_scores.get))
    return result

In [108]:
def laplace_smoothing(self, word, text_class):
  num = self.word_counts[text_class][word] + 1
  denom = self.n_class_items[text_class] + len(self.vocab)
  return math.log(num / denom)

In [109]:
X = train['review'].values
y = train['sentiment'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2) # Ranome seed

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
import numpy as np

# Assuming you have your training data in 'X_train' and 'y_train'

# Preprocess text data
stop_words = 'english'  # Use 'english' as the stop words
vectorizer = CountVectorizer(stop_words=stop_words)
X_train_bow = vectorizer.fit_transform(X_train)

# Train the Multinomial Naive Bayes classifier
mnb = MultinomialNB()
mnb.fit(X_train_bow, y_train)

# Now you have the trained classifier 'mnb' which you can use for prediction.


MultinomialNB()

In [ ]:
from sklearn.metrics import accuracy_score

# Assuming you have your test data in 'X_test' and 'y_test'

# Preprocess test data using the same vectorizer
X_test_bow = vectorizer.transform(X_test)

# Predict using the trained classifier
y_hat = mnb.predict(X_test_bow)

In [ ]:
from sklearn.metrics import accuracy_score
print("Accuracy is ", accuracy_score(y_test, y_hat))

Accuracy is  0.8628


In [ ]:
from sklearn.metrics import classification_report
print("Accuracy is ", classification_report(y_test, y_hat))

Accuracy is                precision    recall  f1-score   support

           0       0.84      0.89      0.86      2466
           1       0.88      0.84      0.86      2534

    accuracy                           0.86      5000
   macro avg       0.86      0.86      0.86      5000
weighted avg       0.86      0.86      0.86      5000

